In [1]:
import os
import sys
from pandas_datareader import data as pdr
import pandas as pd
import fix_yahoo_finance as yf
import importlib
sys.path.append("../MachineLearningStocks/")
from download_historical_prices import *
from parsing_keystats import *
from backtesting import *
from stock_prediction import *
sys.path.append("../awsQLDB/")
os.environ["AWS_PROFILE"] = "Insight"
from create_ledger import *
from connect_to_ledger import *
from create_table import *
from create_index import * 
from insert_document import *
from hashlib import sha256
import pickle

/Users/kenmiyachi/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [2]:
sp500 = build_sp500_dataset()

[*********************100%***********************]  1 of 1 downloaded


In [3]:
sp500_df, stock_df = preprocess_price_data()
parsed_df = parse_keystats(sp500_df, stock_df)

Parsing progress:: 100%|██████████| 560/560 [04:20<00:00,  2.15tickers/s]


In [4]:
backtest()
# check_yahoo()
# current_df = forward()
# current_df.to_csv("forward_sample.csv", index=False)

Classifier performance
Accuracy score:  0.78
Precision score:  0.72

 Stock prediction performance report 
Total Trades: 173
Average return for stock predictions:  38.4 %
Average market return in the same period:  9.3% 
Compared to the index, our strategy earns  29.1 percentage points more


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [5]:
predict_stocks()

13 stocks predicted to outperform the S&P500 by more than 10%:
TMO WGO DISCA FSLR LRCX BLK WY CMI XRX NTAP GTN LMT APD


'TMO WGO DISCA FSLR LRCX BLK WY CMI XRX NTAP GTN LMT APD'

In [6]:
def collectionData(inputData, outHash):
    repo = git.Repo(search_parent_directories=True)
    commit_hash = repo.head.object.hexsha
    collection_data = [
        {
            'GitHash': commit_hash,
            'ScrapeTime': datetime.now(),
            'InputData': inputData,
            'outputHash': outHash
        }
    ]
    return collection_data

def trainingData(inputHash, model):
    repo = git.Repo(search_parent_directories=True)
    commit_hash = repo.head.object.hexsha
    training_data = [
        {
            'GitHash': commit_hash,
            'TrainTime': datetime.now(),
            'InputHash': inputHash,
            'Model': model
        }
    ]
    return training_data

def predictionData(inputHash, model, outHash):
    repo = git.Repo(search_parent_directories=True)
    commit_hash = repo.head.object.hexsha
    prediction_data = [
        {
            'GitHash': commit_hash,
            'InputHash': inputHash,
            'Model': model, 
            'PredictionTime': datetime.now(),
            'OutputHash': outHash
        }
    ]
    return prediction_data

In [7]:
def data_collection(source="Yahoo Finance"):
    sp500 = build_sp500_dataset()
    s = pickle.dumps(sp500)
    outputHash = sha256(s).hexdigest()
    print(outputHash)
    sp500_df, stock_df = preprocess_price_data()
    parse_keystats(sp500_df, stock_df)
    data = collectionData(source, outputHash)
    dynamic_insert("StockMarketPrediction", "Collection", data)


def training(modelName="RandomForest-2020"): 
    model = backtest()
    s = pickle.dumps(model)
    modelHash = sha256(s).hexdigest()
    data = trainingData(modelHash, modelName)
    dynamic_insert("StockMarketPrediction", "Training", data)

def predict(modelName="RandomForest-2020"):
    stats = pd.read_csv('./keystats.csv')
    s = pickle.dumps(stats)
    inputHash = sha256(s).hexdigest()
    prediction = predict_stocks()
    outputHash = sha256(str.encode(prediction)).hexdigest()
    data = predictionData(inputHash, modelName, outputHash)
    dynamic_insert("StockMarketPrediction", "Prediction", data)

In [8]:
data_collection()
training("Test Model 1")
predict("Test Model 1")

[*********************100%***********************]  1 of 1 downloaded
d213a2a575da216befdb349e3bcab9c9fb644d4100eb4e4ed595418961c5e4f5
INFO:insert_document:Inserting some documents in the Collection table...
INFO:insert_document:Documents inserted successfully!
Classifier performance
Accuracy score:  0.78
Precision score:  0.76

 Stock prediction performance report 
Total Trades: 179
Average return for stock predictions:  37.6 %
Average market return in the same period:  9.4% 
Compared to the index, our strategy earns  28.1 percentage points more
INFO:insert_document:Inserting some documents in the Training table...
INFO:insert_document:Documents inserted successfully!
13 stocks predicted to outperform the S&P500 by more than 10%:
TMO WGO DISCA FSLR LRCX BLK WY CMI XRX NTAP GTN LMT APD
INFO:insert_document:Inserting some documents in the Prediction table...
INFO:insert_document:Documents inserted successfully!


In [38]:
print(b'Nice')

b'Nice'
